In [2]:
from matplotlib.dates import date2num, num2date
from matplotlib.colors import ListedColormap
from matplotlib import dates as mdates
from matplotlib.patches import Patch
from matplotlib import pyplot as plt
from matplotlib import ticker

import seaborn as sns


from global_config import config
import pandas as pd
import numpy as np
import rpy2
import os

from datetime import date, timedelta


data_dir_mnps   = config.get_property('geo_dir')
data_dir        = config.get_property('data_dir')
fb_ppl_data     = config.get_property('covid_fb')
results_dir     = config.get_property('results_dir')
data_cases_path = os.path.join(fb_ppl_data, 'agglomerated', 'geometry')


In [7]:

# Defines lockdowns
no_lock_lab = "No Lockdown Imposed"
lock_lab    = "Lockdown Imposed"


lockdowns = []
lockdowns.append({"code": "A",
                  "start_date" : pd.to_datetime("2020-07-13"),
                  "end_date"   : pd.to_datetime("2020-07-23"),
                  "places":["chapinero",
                            "los_martires",
                            "san_cristobal",
                            "tunjuelito",
                            "rafel_uribe_uribe",
                            "ciudad_bolivar",
                            "santa_fe",
                            "usme"]})

lockdowns.append({"code": "B",
                  "start_date" : pd.to_datetime("2020-07-23"),
                  "end_date"   : pd.to_datetime("2020-08-06"),
                  "places":["bosa",
                            "antonio_narino",
                            "kennedy",
                            "puente_aranda",
                            "fontibon"]})

lockdowns.append({"code": "C",
                  "start_date" : pd.to_datetime("2020-07-31"),
                  "end_date"   : pd.to_datetime("2020-08-14"),
                  "places":["suba",
                            "engativa",
                            "barrios_unidos"]})

lockdowns.append({"code": "D",
                  "start_date" : pd.to_datetime("2020-08-16"),
                  "end_date"   : pd.to_datetime("2020-08-27"),
                  "places":["usaquen",
                            "chapinero",
                            "santa_fe",
                            "candelaria",
                            "teusaquillo",
                           "puente_aranda",
                           "antonio_narino"]})


In [12]:
dict_correct = {'Los Martires': 'Los Mártires', 'Fontibon': 'Fontibón', 'Engativa': 'Engativá',
                            'San Cristobal': 'San Cristóbal', 'Usaquen': 'Usaquén',
                            'Ciudad Bolivar': 'Ciudad Bolívar', 'Candelaria': 'La Candelaria'}

# load Rt locality estimates
cases_df = pd.read_csv(os.path.join(data_cases_path, 'cases.csv'), parse_dates=["date_time"])


cases_df["poly_id"]  = cases_df["poly_id"].apply(lambda s:   s.replace("colombia_bogota_localidad_",""))

cases_df["poly_name"]  = cases_df["poly_id"].apply(lambda s:   ' '.join( [word.capitalize() for word in s.replace("colombia_bogota_localidad_","").split('_') ] ) )
cases_df["poly_name"]  = cases_df["poly_name"].replace( dict_correct )

cases_bog_df = cases_df.groupby(['poly_id', 'poly_name', 'date_time']).sum()[["num_cases", "num_diseased", "num_infected_in_icu"]]
cases_bog_df = cases_bog_df.unstack([0,1]).resample('1D').sum().stack().stack().reset_index().rename(columns={'date_time':'date','num_cases': 'confirm',
                                                                                                                'num_diseased': 'deaths',
                                                                                                                'num_infected_in_icu': 'icu'})


rt_df_all = []
path_to_save = results_dir

for poly_name in cases_bog_df["poly_id"].unique():
    rt_loc = pd.read_csv(os.path.join(path_to_save, 'rt', 'rt_df_{}_confirmation.csv'.format(poly_name)))
    rt_loc["location_id"] = poly_name
    rt_loc = rt_loc[rt_loc.type=='estimate']
    rt_loc = rt_loc[rt_loc.variable=='R']
    rt_loc['date'] = rt_loc['date'].map(lambda x: pd.to_datetime(0)+timedelta(days=x))
    rt_loc = rt_loc.drop(columns=['Unnamed: 0'])
    rt_df_all.append(rt_loc)

rt_df_all = pd.concat(rt_df_all)
rt_df_all


,date,variable,strat,type,median,mean,sd,lower_90,lower_50,lower_20,upper_20,upper_50,upper_90,location_id
0,2020-03-14,R,NA_character_,estimate,1.776741,1.788429,0.164820,1.531016,1.680119,1.739704,1.820044,1.892763,2.079742,antonio_narino
1,2020-03-15,R,NA_character_,estimate,1.699241,1.707016,0.150385,1.476043,1.605850,1.663997,1.733912,1.801143,1.960841,antonio_narino
2,2020-03-16,R,NA_character_,estimate,1.616431,1.624819,0.144584,1.407748,1.525607,1.583536,1.654101,1.716474,1.870811,antonio_narino
3,2020-03-17,R,NA_character_,estimate,1.536636,1.543448,0.143658,1.322035,1.441880,1.500192,1.573392,1.633716,1.794197,antonio_narino
4,2020-03-18,R,NA_character_,estimate,1.457039,1.464396,0.144341,1.235902,1.363457,1.422957,1.494264,1.555988,1.716472,antonio_narino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2021-04-14,R,NA_character_,estimate,0.512427,0.513533,0.046124,0.439498,0.481247,0.501368,0.524696,0.545360,0.589619,usme
397,2021-04-15,R,NA_character_,estimate,0.469276,0.470670,0.048098,0.394049,0.437357,0.457306,0.482133,0.504178,0.550368,usme
398,2021-04-16,R,NA_character_,estimate,0.431838,0.433195,0.050039,0.353642,0.398515,0.419064,0.444571,0.468408,0.516545,usme
399,2021-04-17,R,NA_character_,estimate,0.399098,0.400776,0.051964,0.319283,0.364770,0.386441,0.412151,0.437311,0.487248,usme


In [27]:
# prelockdown period
pre_lockdown_start = pd.to_datetime("2020-06-20")
pre_lockdown_end   = pd.to_datetime("2020-07-12")

lockdown_start     = pd.to_datetime("2020-07-13")
lockdown_end       = pd.to_datetime("2020-08-27")

# poslockdown period
pos_lockdown_start = pd.to_datetime("2020-08-28")
pos_lockdown_end   = pd.to_datetime("2020-09-15")


# Assigns moments
rt_df_all.loc[(rt_df_all.date <= pre_lockdown_end)
            & (rt_df_all.date >= pre_lockdown_start),'lockdown_type'] = "pre-lockdown1"

rt_df_all.loc[(rt_df_all.date <= lockdown_end)
            & (rt_df_all.date >= lockdown_start),'lockdown_type'] = "lockdown1"

rt_df_all.loc[(rt_df_all.date <= pos_lockdown_end)
            & (rt_df_all.date >= pos_lockdown_start),'lockdown_type'] = "pos-lockdown1"


rt_df_all["lockdown_group"] = 'None'
rt_df_all["locked_down"]    = False

for lock_info in lockdowns:
    g_lockdown_start     = lock_info["start_date"]
    g_lockdown_end       = lock_info["end_date"]
    rt_df_all.loc[ (rt_df_all.date <= g_lockdown_end)
                    & (rt_df_all.date >= g_lockdown_start), 'lockdown_group'] += "-"+lock_info["code"]


    rt_df_all.loc[ (rt_df_all.date <= g_lockdown_end)
                    & (rt_df_all.date >= g_lockdown_start) & (rt_df_all.location_id.isin(lock_info["places"]) ), 'locked_down'] = True

rt_pivoted_df = pd.pivot_table(rt_df_all, values='median', index=['date', 'lockdown_type', 'lockdown_group', "locked_down"], columns=['location_id']).fillna(0).reset_index()
rt_pivoted_df 


location_id,date,lockdown_type,lockdown_group,locked_down,antonio_narino,barrios_unidos,bosa,candelaria,chapinero,ciudad_bolivar,...,los_martires,puente_aranda,rafael_uribe_uribe,san_cristobal,santa_fe,suba,teusaquillo,tunjuelito,usaquen,usme
0,2020-06-20,pre-lockdown1,None,False,1.137485,1.267941,1.135954,0.987836,1.278688,1.150172,...,1.045485,1.122430,1.215726,1.195448,1.221898,1.165110,1.242313,1.218197,1.241528,1.157752
1,2020-06-21,pre-lockdown1,None,False,1.130961,1.259943,1.137648,0.968994,1.266584,1.141718,...,1.040446,1.125629,1.204959,1.179015,1.198945,1.181418,1.228636,1.210117,1.242168,1.147632
2,2020-06-22,pre-lockdown1,None,False,1.121956,1.250843,1.139657,0.957143,1.251811,1.132121,...,1.037916,1.126349,1.192918,1.161678,1.172914,1.199091,1.214950,1.202297,1.243439,1.138064
3,2020-06-23,pre-lockdown1,None,False,1.111100,1.241969,1.140551,0.952125,1.234769,1.123279,...,1.035759,1.124711,1.183151,1.144367,1.146591,1.219765,1.204304,1.194515,1.244231,1.129652
4,2020-06-24,pre-lockdown1,None,False,1.098098,1.231430,1.139749,0.952901,1.218061,1.114891,...,1.035823,1.122279,1.173054,1.125986,1.124302,1.238675,1.193959,1.185826,1.242715,1.124131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2020-09-11,pos-lockdown1,None,False,0.934203,1.024863,0.990154,1.139314,1.136164,0.945876,...,0.989528,1.073954,0.957144,0.957069,1.010590,1.117127,1.216318,0.996298,1.168801,0.981537
129,2020-09-12,pos-lockdown1,None,False,0.937391,1.018485,1.003631,1.149738,1.136376,0.949061,...,0.989242,1.085897,0.965914,0.967488,1.013773,1.123942,1.217238,0.998367,1.180739,0.990548
130,2020-09-13,pos-lockdown1,None,False,0.944160,1.013393,1.015836,1.154913,1.134027,0.953817,...,0.990994,1.094663,0.974823,0.977537,1.015323,1.132745,1.211758,1.000037,1.186884,0.998177
131,2020-09-14,pos-lockdown1,None,False,0.952658,1.008469,1.026994,1.153344,1.128394,0.959304,...,0.991307,1.100905,0.984743,0.986818,1.017580,1.139905,1.202992,1.001115,1.188222,1.005922


In [25]:
rt_df_all

Timestamp('2020-06-20 00:00:00')